In [111]:
pip install scikit-learn


[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [112]:
import pycrfsuite

from sklearn.preprocessing import LabelBinarizer
from itertools import chain
from sklearn.metrics import classification_report

def readCorpus(fpath):
    sents = []
    with open(fpath) as fd:
        sent = []
        for l in fd:
            #lt = l.strip().decode("utf8")
            lt = l.strip()
            if not lt:
                sents.append(sent)
                sent = []
            else:
                w_t = lt.split('\t')
                sent.append([w_t[0], w_t[1]])
    return sents

def word2features(sent,i):
    word = sent[i][0]
    print(word)
    postag = sent[i][1]
    features = [
        #'bias',
        'word.lower=' + word.lower(),
        #'word[-3:]=' + word[-3:],
        #'word[-2:]=' + word[-2:],
        #'word.isupper=%s' % word.isupper(),
        #'word.istitle=%s' % word.istitle(),
        #'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        #'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        pass
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            #'-1:word.istitle=%s' % word1.istitle(),
            #'-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            #'-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        pass
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            #'+1:word.istitle=%s' % word1.istitle(),
            #'+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            #'+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, label in sent]
def sent2tokens(sent):
    return [token for token, label in sent]

In [113]:
train_sents = readCorpus("CrfTrain.TXT")
#print(train_sents)
x_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]
#print(y_train)

print("start append train set.")
trainer = pycrfsuite.Trainer(verbose=False)
for xseq, yseq in zip(x_train, y_train):
    trainer.append(xseq, yseq)
print("append train set done.")
trainer.set_params({
    'c1': 1.0,
    'c2': 1e-3,
    'max_iterations': 500,
    'feature.possible_transitions': True
    })

trainer.train("trained_model")
test_object = pycrfsuite.Tagger()
test_object.open("trained_model")


same
as
picture
but
string
chahi
ali
moto
cha
but
nice
quality
kapada
ta
ramrai
lagyo
but
size
thulo
bhayo
kapada
ma
size
ko
tag
pani
chahina
kasto
ho
sabai
lai
same
size
pathaune
bhaye
kina
size
mention
gareko
ho
k
aba
satna
ni
aau
jau
time
waste
products
ta
ramrai
raicha
tara
pwal
raicha
hai
yesma
euta
please
return
it
arko
send
gardinuna
hai
poor
quality
of
product
actually
it's
zero
star
for
it
plastic
ko
body
cha
looks
cheap
as
well
its
better
to
buy
a
premium
watch
with
a
little
more
money
it's
really
very
nice
but
moto
cha
bhane
blouse
chahi
pugdaina
hai
overall
it's
damn
good
and
quality
is
also
nice
according
to
the
price
saree
dherai
ramro
lagyo
malai
color
pani
ekdam
ramro
lagyo
saree
last
naramro
lagyo
i
am
angry
not
as
expected
color
ali
changed
cha
price
anusar
ramro
cha
daraz
ghatiya
online
shopping
websites
ho
diyeko
huncha
kehi
delivery
garcha
arko
saman
jaba
1
year
ko
warranty
cha
bhane
card
kaha
harai
halcha
ramro
sanga
delivery
garna
audaina
bhane
kina
byapar
garcha

In [114]:
#Test on the first sentence of the training data
#First load the trained model
tagger = pycrfsuite.Tagger()
tagger.open('trained_model')

#Read the first sentence
#print(train_sents[0])
example_sent = train_sents[0]
print(' '.join(sent2tokens(example_sent)), end='\n\n')
print(sent2labels(example_sent))

#Predict and test
print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

#This will give you the precision, recall, F-score
def sentiment_classification_report(y_true, y_pred):
 
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

##Evaluation of the labeller
y_pred = [tagger.tag(xseq) for xseq in x_train]
print(sentiment_classification_report(y_train, y_pred))

same as picture but string chahi ali moto cha but nice quality

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ENG', 'O']
same
as
picture
but
string
chahi
ali
moto
cha
but
nice
quality
Predicted: O O O O O O O O O O B-ENG O
Correct:   O O O O O O O O O O B-ENG O
              precision    recall  f1-score   support

       B-ENG       1.00      1.00      1.00       266
       I-ENG       1.00      1.00      1.00        84
       B-NEG       1.00      1.00      1.00        62
       I-NEG       1.00      1.00      1.00        10
       B-POS       1.00      1.00      1.00       211
       I-POS       1.00      1.00      1.00        58
           o       1.00      1.00      1.00         2

   micro avg       1.00      1.00      1.00       693
   macro avg       1.00      1.00      1.00       693
weighted avg       1.00      1.00      1.00       693
 samples avg       0.21      0.21      0.21       693



e:\Project Sentiment analysis\projenv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
e:\Project Sentiment analysis\projenv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [115]:
def read_text_file(file_path):
    words=[]  # List to store the words
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace
            line_words = line.split()  # Split line into individual words
            for word in line_words:
                words.append(word)  # Add words to the list

    
    return words


file_path = "E:/Project Sentiment analysis/CRF/xyz.txt"  # Path to the text file

# Read the text file and store words in a list
word_list = read_text_file(file_path)
print(word_list)



['kasto', 'naramro', 'product', 'yo', 'i', 'dont', 'recommend', 'it']


In [116]:
#Test on the test data 
#First load the trained model
tagger = pycrfsuite.Tagger()
tagger.open('trained_model')

#Predict and test
print(word_list)

new_list = [[word, ''] for word in word_list]

# example=[]
# example.append(("hi"))
# print(example)

feature=sent2features(new_list)
#print(feature)
lab=tagger.tag(feature)
print(lab)
print("Predicted:", ' '.join(lab))

PosCount=0
NegCount=0
for i in lab:
    if(i=="B-POS" or i=="I-POS"):
        PosCount+=1
    elif(i=="B-NEG" or i=="I-NEG"):
        NegCount+=1
print(f"Positive Sentiment Count={PosCount}")
print(f"Negative Sentiment Count={NegCount}")

['kasto', 'naramro', 'product', 'yo', 'i', 'dont', 'recommend', 'it']
kasto
naramro
product
yo
i
dont
recommend
it
['B-POS', 'I-POS', 'B-POS', 'I-POS', 'B-POS', 'I-POS', 'B-POS', 'I-POS']
Predicted: B-POS I-POS B-POS I-POS B-POS I-POS B-POS I-POS
Positive Sentiment Count=8
Negative Sentiment Count=0
